In [97]:
import pandas as pd
import json
import numpy as np


In [13]:
checkin_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_checkin.json"
business_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json"
review_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"
tip_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_tip.json"
user_path="/home/jaume/Escritorio/Ultim_Semestre/Comp.ML/Practicas/project/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_user.json"


In [14]:
'''
with open(checkin_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

checkin = pd.DataFrame(data)
with open(business_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

business = pd.DataFrame(data)'''
with open(review_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

review = pd.DataFrame(data)
'''
with open(tip_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

tip = pd.DataFrame(data)
with open(user_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

user = pd.DataFrame(data)
'''

'\nwith open(tip_path, "r", encoding="utf-8") as f:\n    data = [json.loads(line) for line in f]\n\ntip = pd.DataFrame(data)\nwith open(user_path, "r", encoding="utf-8") as f:\n    data = [json.loads(line) for line in f]\n\nuser = pd.DataFrame(data)\n'

In [151]:
review=review.head(150)
review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
145,pJRn59F_lyNO1zT3TCVd0Q,TGgfqWnUaCf6DM7TLuNhDQ,pSmOH4a3HNNpYM82J5ycLA,3.0,0,0,0,"This is a pretty good pancake place, but it's ...",2018-03-04 00:07:27
146,WSeO-PP8yxmUs8ts2myRzA,W7c1eS1WpxGC5295iJpuTQ,RfkaKLn494TKC943fJhKOw,5.0,1,0,0,Great finds and wonderful staff! \nLOVE our fi...,2013-09-04 14:41:57
147,ypFqmURIY41F4pWaMW1VrQ,hToW2eoTAYlOcuNz7I73Cg,7L1kXfwU5XM6f6Jxo_L7Yg,5.0,1,0,0,We've been coming here since I was a kid. The ...,2015-07-03 22:36:04
148,HME_ksGph3se7Aze5hxa-Q,kSMOJwJXuEUqzfmuFncK4A,kxX2SOes4o-D3ZQBkiMRfA,2.0,0,0,1,Dine-in gets 2 stars. Disappointing service & ...,2014-07-13 17:25:47


In [152]:
review_short= review.drop(columns=["user_id","business_id","date"])
review_short

,review_id,stars,useful,funny,cool,text
0,KU_O5udG6zpxOg-VcAEodg,3.0,0,0,0,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,5.0,1,0,1,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...
...,...,...,...,...,...,...
145,pJRn59F_lyNO1zT3TCVd0Q,3.0,0,0,0,"This is a pretty good pancake place, but it's ..."
146,WSeO-PP8yxmUs8ts2myRzA,5.0,1,0,0,Great finds and wonderful staff! \nLOVE our fi...
147,ypFqmURIY41F4pWaMW1VrQ,5.0,1,0,0,We've been coming here since I was a kid. The ...
148,HME_ksGph3se7Aze5hxa-Q,2.0,0,0,1,Dine-in gets 2 stars. Disappointing service & ...


### Lowecasing

In [153]:
review_lower=review_short
review_lower["text"]=review_lower["text"].str.lower()
review_lower

,review_id,stars,useful,funny,cool,text
0,KU_O5udG6zpxOg-VcAEodg,3.0,0,0,0,"if you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,5.0,1,0,1,i've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,3.0,0,0,0,family diner. had the buffet. eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,5.0,1,0,1,"wow! yummy, different, delicious. our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,1,0,1,cute interior and owner (?) gave us tour of up...
...,...,...,...,...,...,...
145,pJRn59F_lyNO1zT3TCVd0Q,3.0,0,0,0,"this is a pretty good pancake place, but it's ..."
146,WSeO-PP8yxmUs8ts2myRzA,5.0,1,0,0,great finds and wonderful staff! \nlove our fi...
147,ypFqmURIY41F4pWaMW1VrQ,5.0,1,0,0,we've been coming here since i was a kid. the ...
148,HME_ksGph3se7Aze5hxa-Q,2.0,0,0,1,dine-in gets 2 stars. disappointing service & ...


In [154]:
corpus=review_lower.drop(columns=["useful","funny","cool"])

### Lemmatization

In [155]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [156]:
corpus

,review_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,3.0,"if you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,5.0,i've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,3.0,family diner. had the buffet. eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,5.0,"wow! yummy, different, delicious. our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,cute interior and owner (?) gave us tour of up...
...,...,...,...
145,pJRn59F_lyNO1zT3TCVd0Q,3.0,"this is a pretty good pancake place, but it's ..."
146,WSeO-PP8yxmUs8ts2myRzA,5.0,great finds and wonderful staff! \nlove our fi...
147,ypFqmURIY41F4pWaMW1VrQ,5.0,we've been coming here since i was a kid. the ...
148,HME_ksGph3se7Aze5hxa-Q,2.0,dine-in gets 2 stars. disappointing service & ...


In [157]:
corpus['words'] = None
for index, sample in corpus.iterrows():
    list_words = [] 
    sent = nlp(sample.text)
    
    for token in sent:
        list_words.append(token.lemma_)
    
    corpus.at[index, "words"] = list_words 


### Remove Stopwords

In [158]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
corpus["words"] = corpus["words"].apply(lambda x: [word for word in x if word.lower() not in stop_words])



[nltk_data] Downloading package stopwords to /home/jaume/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [159]:
corpus

,review_id,stars,text,words
0,KU_O5udG6zpxOg-VcAEodg,3.0,"if you decide to eat here, just be aware it is...","[decide, eat, ,, aware, go, take, 2, hour, beg..."
1,BiTunyQ73aT9WBnpR9DZGw,5.0,i've taken a lot of spin classes over the year...,"[take, lot, spin, class, year, ,, nothing, com..."
2,saUsX_uimxRlCVr67Z4Jig,3.0,family diner. had the buffet. eclectic assortm...,"[family, diner, ., buffet, ., eclectic, assort..."
3,AqPFMleE6RsU23_auESxiA,5.0,"wow! yummy, different, delicious. our favo...","[wow, !, , yummy, ,, different, ,, , delicio..."
4,Sx8TMOWLNuJBWer-0pcmoA,4.0,cute interior and owner (?) gave us tour of up...,"[cute, interior, owner, (, ?, ), give, tour, u..."
...,...,...,...,...
145,pJRn59F_lyNO1zT3TCVd0Q,3.0,"this is a pretty good pancake place, but it's ...","[pretty, good, pancake, place, ,, little, diso..."
146,WSeO-PP8yxmUs8ts2myRzA,5.0,great finds and wonderful staff! \nlove our fi...,"[great, find, wonderful, staff, !, \n, love, f..."
147,ypFqmURIY41F4pWaMW1VrQ,5.0,we've been coming here since i was a kid. the ...,"[come, since, kid, ., staff, watch, kid, grow,..."
148,HME_ksGph3se7Aze5hxa-Q,2.0,dine-in gets 2 stars. disappointing service & ...,"[dine, -, get, 2, star, ., disappointing, serv..."


In [160]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus["text"]).toarray()

print(X.shape)

(150, 2459)


In [161]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### VADER POLARITY

In [162]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [163]:
y_pred = []
for rev in corpus["text"]:
    y_pred.append(1 if analyzer.polarity_scores(rev)['compound'] > 0 else 0)

In [164]:
corpus["vader"]= y_pred
corpus["text"][corpus["vader"]==0]

8      this easter instead of going to lopez lake we ...
27     service was crappy, and food was mediocre.  i ...
41     straight to the point, it's cheap, it tastes a...
53     i've been to this location many times when i l...
64     the tv shows are $4.99 and they have commercia...
73     if i could give it a zero, i would. i order a ...
84     skip this train wreck if you are looking for d...
85     meh is all i can say for tds. the restaurant i...
98     service and management terrible... after messi...
101    been to this location twice and will never go ...
103    federal doughnuts is a new concession at the b...
120    omg!!! i can't believe this kind of service ex...
123    say you decide that you need a new door...and ...
124    worst place i've ever visited. the bouncer was...
126    so disappointing on so many levels. have been ...
132    men's clubs is the tits, plain and simple. all...
148    dine-in gets 2 stars. disappointing service & ...
Name: text, dtype: object

In [165]:
corpus["text"].to_numpy()

array(["if you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. we have tried it multiple times, because i want to like it! i have been to it's other locations in nj and never had a bad experience. \n\nthe food is good, but it takes a very long time to come out. the waitstaff is very young, but usually pleasant. we have just had too many experiences where we spent way too long waiting. we usually opt for another diner or restaurant on the weekends, in order to be done quicker.",
       "i've taken a lot of spin classes over the years, and nothing compares to the classes at body cycle. from the nice, clean space and amazing bikes, to the welcoming and motivating instructors, every class is a top notch work out.\n\nfor anyone who struggles to fit workouts in, the online scheduling system makes it easy to plan ahead (and there's no need to line up way in advanced like many gyms make you do).\n\nthere is no way i can write this review without givin

In [166]:
corpus['words_join'] = None
corpus["words_join"] = corpus["words"].apply(lambda x: " ".join([word for word in x if word]))

## LDA

In [167]:
from sklearn.feature_extraction.text import CountVectorizer
n_topics = 5
# feature extraction
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus["words_join"].to_numpy())


In [168]:
X

<150x1951 sparse matrix of type '<class 'numpy.int64'>'
	with 5502 stored elements in Compressed Sparse Row format>

In [169]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=n_topics, learning_method='online', 
                                          random_state=0, verbose=0)
lda_topic_matrix = lda_model.fit_transform(X)

In [170]:
from collections import Counter

# Define helper functions
def get_keys(topic_matrix):
    '''
    returns an integer list of predicted topic 
    categories for a given topic matrix
    '''
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def keys_to_counts(keys):
    '''
    returns a tuple of topic categories and their 
    accompanying magnitudes for a given list of keys
    '''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)


In [171]:
lda_keys = get_keys(lda_topic_matrix)
lda_categories, lda_counts = keys_to_counts(lda_keys)

In [172]:
# Define helper functions
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''
    returns a list of n_topic strings, where each string contains the n most common 
    words in a predicted category, in order
    '''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        print(temp_vector_sum)
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
        
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [173]:
lda_keys

[1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 3,
 0,
 4,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 4,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 4,
 3,
 1,
 4,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [174]:
top_n_words_lda = get_top_n_words(10, lda_keys, X, vectorizer)

for i in range(len(top_n_words_lda)):
    print("Topic {}: ".format(i+1), top_n_words_lda[i])

  (0, 1015)	1
  (0, 1384)	1
  (0, 1294)	1
  (0, 1063)	1
  (0, 1114)	1
  (0, 952)	1
  (0, 734)	1
  (0, 193)	3
  (0, 130)	2
  (0, 566)	1
  (0, 1411)	2
  (0, 126)	1
  (0, 196)	1
  (0, 1831)	1
  (0, 1424)	1
  (0, 495)	1
  (0, 769)	1
  (0, 960)	1
  (0, 1009)	1
  (0, 1738)	1
  (0, 817)	1
  (0, 1122)	1
  (0, 1180)	1
  (0, 610)	1
  (0, 388)	1
  :	:
  (0, 272)	1
  (0, 1137)	1
  (0, 1752)	1
  (0, 1166)	2
  (0, 1578)	1
  (0, 1438)	1
  (0, 191)	1
  (0, 812)	1
  (0, 1789)	1
  (0, 1344)	1
  (0, 300)	1
  (0, 1627)	1
  (0, 648)	1
  (0, 1135)	2
  (0, 226)	1
  (0, 393)	1
  (0, 467)	1
  (0, 592)	1
  (0, 896)	1
  (0, 1715)	1
  (0, 1767)	1
  (0, 838)	1
  (0, 1879)	1
  (0, 13)	1
  (0, 909)	1
  (0, 569)	1
  (0, 1094)	1
  (0, 445)	1
  (0, 267)	1
  (0, 711)	1
  (0, 1943)	1
  (0, 1290)	1
  (0, 1809)	1
  (0, 523)	1
  (0, 1265)	1
  (0, 148)	1
  (0, 1759)	1
  (0, 1101)	1
  (0, 231)	1
  (0, 1844)	1
  (0, 339)	1
  (0, 114)	1
  (0, 1192)	1
  (0, 1004)	1
  (0, 826)	1
  (0, 463)	1
  (0, 450)	1
  (0, 604)	1
  (0, 1903)	

In [ ]:
corpus2=corpus
texts=corpus["words"].tolist()

[[(0, 1),
  (1, 1),
  (2, 5),
  (3, 6),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 2),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 2),
  (33, 2),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1)],
 [(0, 2),
  (1, 1),
  (2, 10),
  (3, 6),
  (19, 1),
  (22, 1),
  (32, 1),
  (39, 2),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 5),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),


In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
# Crear diccionario y corpus
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Rango de número de tópicos a probar
topic_range = range(2, 11)
coherence_values = []

# Entrenar modelos LDA y calcular coherencia
for num_topics in topic_range:
    lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherence_model.get_coherence())

# Graficar coherencia vs. número de tópicos
plt.figure(figsize=(8, 5))
plt.plot(topic_range, coherence_values, marker='o')
plt.xlabel("Número de Tópicos")
plt.ylabel("Coherencia")
plt.title("Selección del Número de Tópicos con Coherencia")
plt.grid()
plt.show()

TypeError: list indices must be integers or slices, not str